# RAG Post Retrieval Optimization - Reranking featuring Bedrock and llamaindex

- Vector Database (Faiss / local on the notebook for this demo)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- ReRanking Model (Claude Haiku) running on Bedrock
- Llamaindex for orchestration (ingestion, reranking, retrieval and final response synthesis)
- Datasets (Amazon SEC 10-k statements for year 2022 and 2023 )

<img src="reranking-bedrock.png" width="800" height="400">

In [1]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-embeddings-instructor
%pip install llama-index-postprocessor-rankgpt-rerank

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.8.2
Uninstalling pydantic-2.8.2:
  Successfully uninstalled pydantic-2.8.2
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 2.14.1 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings

In [6]:
import json
from typing import Sequence, List
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

In [7]:
!mkdir -p 'data/amazon/'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf' -O 'data/amazon/amazon_2023.pdf'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf' -O 'data/amazon/amazon_2022.pdf'

--2024-08-15 21:48:41--  https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.3, 68.70.205.1, 68.70.205.2, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘data/amazon/amazon_2023.pdf’

100%[======================================>] 800,598     --.-K/s   in 0.05s   

2024-08-15 21:48:41 (16.5 MB/s) - ‘data/amazon/amazon_2023.pdf’ saved [800598/800598]

--2024-08-15 21:48:41--  https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.4, 68.70.205.2, 68.70.205.1, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712683 (696K) [application/pdf]
Saving to: ‘data/amazon/amazon_2022.pdf’

100%[===

In [8]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="./data/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

In [9]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", -1)


def get_retrieved_nodes(
    query_str, vector_top_k=5, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=amazon_index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=Bedrock(model = "anthropic.claude-3-haiku-20240307-v1:0"),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

/tmp/ipykernel_16796/1899260342.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [10]:
new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    with_reranker=False,
)

In [11]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.731064,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations—Overview,” and in the risk factorsbelow, global economic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many ofthe risks discussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a materialadverse effect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure better terms from vendors, adopt more aggressive pricing, and devote more resources to technology, infrastructure,fulfillment, and marketing.Competition continues to intensify, including with the development of new business models and the entry of new and well-funded competitors, and asour competitors enter into business combinations or alliances and established companies in other market segments expand to become competitive with ourbusiness. In addition, new and enhanced technologies, including search, web and infrastructure computing services, digital content, and electronic devicescontinue to increase our competition. The Internet facilitates competitive entry and comparison shopping, which enhances the ability of new, smaller, or lesserknown businesses to compete against us. As a result of competition, our product and service offerings may not be successful, we may fail to gain or may losebusiness, and we may be required to increase our spending or lower prices, any of which could materially reduce our sales and profits."
1,0.725057,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations,” and in the risk factors below, globaleconomic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many of the risksdiscussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a material adverseeffect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure bette

In [15]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    reranker_top_n=5,
    with_reranker=True,
)

After Reranking, new rank list for nodes: [2, 5, 7, 4, 9, 3, 8, 6, 0, 1]

In [16]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.710425,"which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact.We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand and consumer spending patterns or otherwise optimize and operate our fulfillment network and datacenters successfully from time to time result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairmentcharges, any of which could materially harm our business. As we continue to add fulfillment and data center capability or add new businesses with differentrequirements, our fulfillment and data center networks become increasingly complex and operating them becomes more challenging. There can be no assurancethat we will be able to operate our networks effectively.10"
1,0.680842,"We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand or otherwise optimize and operate our fulfillment network and data centers successfully from time totime result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairment charges, any of which couldmaterially harm our business. As we continue to add fulfillment and data center capability or add new businesses with different requirements, our fulfillmentand data center networks become increasingly complex and operating them becomes more challenging. There can be no assurance that we will be able tooperate our networks effectively.In addition, failure to optimize inventory or staffing in our fulfillment network increases our net shipping cost by requiring long-zone or partialshipments. We and our co-sourcers may be unable to adequately staff our fulfillment network and customer service centers. For example, productivity acrossour fulfillment network currently is being affected by regional labor market and global supply chain constraints, which increase payroll costs and make itdifficult to hire, train, and deploy a sufficient number of people to operate our fulfillment network as efficiently as we would like.Under some of our commercial agreements, we maintain the inventory of other companies, thereby increasing the complexity of tracking inventory andoperating our fulfillment network. Our failure to properly handle such inventory or the inability of the other businesses on whose behalf we perform inventoryfulfillment services to accurately forecast product demand may result in us being unable to secure sufficient storage space or to optimize our fulfillmentnetwork or cause other unexpected costs and other harm to our business and reputation.We rely on a limited number of shipping companies to deliver inventory to us and completed orders to our customers. An inability to negotiate acceptableterms with these companies or performance problems,

# Seeing reranking in action in final results

In [17]:
query_engine_naive = amazon_index.as_query_engine(similarity_top_k=10)

In [18]:
response = query_engine_naive.query(
    "Describe key business risks for Amazon during covid"
)

In [19]:
print(response)

Based on the context provided, some key business risks Amazon faced during the COVID-19 pandemic include:

1. Disruptions from the public health crisis, which could impact Amazon's operations, supply chain, fulfillment network, and ability to meet customer demand.

2. Challenges in optimizing and operating their fulfillment network and data centers due to factors like labor constraints, supply chain issues, and rapidly changing customer demand patterns during the pandemic.

3. Potential system interruptions or delays caused by events related to the pandemic, preventing Amazon from efficiently accepting or fulfilling orders and providing services.

4. Risks related to their supplier relationships, such as limited or single sources of supply, which could be impacted by the pandemic and result in shortages or delays in obtaining merchandise, content, components or services.

5. Difficulties in hiring, training, and retaining sufficient personnel to operate their stores, fulfillment networ

In [21]:
reranker = RankGPTRerank(
            llm=Bedrock(model = "anthropic.claude-3-haiku-20240307-v1:0"),
            top_n=5,
            verbose=True,
        )

In [22]:
query_engine_rerank = amazon_index.as_query_engine(similarity_top_k=10, node_postprocessor =[reranker])

In [23]:
response = query_engine_rerank.query(
    "Describe key business risks for Amazon during covid"
)

In [24]:
print(response)

Based on the context provided, some key business risks for Amazon during the COVID-19 pandemic include:

1. Disruptions from natural or human-caused disasters, including public health crises like COVID-19. The pandemic likely caused disruptions to Amazon's operations, supply chains, and workforce.

2. Challenges in optimizing and operating their fulfillment network and data centers due to fluctuations in customer demand, labor shortages, and supply chain constraints during the pandemic. This could lead to excess or insufficient capacity, service interruptions, and increased costs.

3. Potential negative impacts on their supplier relationships, as suppliers faced their own challenges during COVID-19. This risked product/service shortages or delays if suppliers limited or stopped providing merchandise, content, components, or services to Amazon.

4. Difficulties in hiring, training, retaining, and managing sufficient personnel to operate stores, fulfillment centers, and data centers amid